In [1]:
# Importing libraries
import cv2 
import numpy as np

In [14]:
class Board:
    
    def __init__(self):
        self.mouse_pressed = False
        self.board = np.zeros(shape=(1024, 1024, 3), dtype=np.uint8)
        self.prev= np.zeros(shape=(1024, 1024, 3), dtype=np.uint8)
        self.char_color = (255, 255, 255)
        self.count= -1
        
    def reset(self):
        # reset image
        self.board = np.zeros((1024, 1024, 3), np.uint8)
        
    def draw(self):
        """
        Method to draw multiple Charactes on a single Board
        """

        self.reset()

        window_name = 'Draw'

        cv2.namedWindow(winname=window_name)
        cv2.setMouseCallback(window_name=window_name, on_mouse=self.mouse_callback)

        # Until 'ESC' key is pressed
        while True:
            cv2.imshow(winname=window_name, mat=self.board)

            # When ESC key pressed
            k = cv2.waitKey(delay=1) & 0xFF
            if k == 27:
                break

        cv2.destroyAllWindows()
    def get_contours(self):
        """
        Method to find contours in an image and crop them and return a list with cropped contours
        """
               
        if self.count==0:
            main_image = self.img
        else:
            main_image= self.img- self.prev
        
        cv2.imshow('impa', main_image)
        self.prev= self.img.copy()
        
        
        # convert to greyscale and apply Gaussian filtering
        main_image = cv2.cvtColor(src=main_image, code=cv2.COLOR_BGR2GRAY)
        main_image = cv2.GaussianBlur(src=main_image, ksize=(5, 5), sigmaX=0)

        # threshold the image
        _, main_image = cv2.threshold(src=main_image, thresh=127, maxval=255, type=cv2.THRESH_BINARY)
        
        # find contours in the image
        contours, _ = cv2.findContours(image=main_image.copy(), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)

        # get rectangles containing each contour
        bboxes=[]
        

        bboxes = [cv2.boundingRect(array=contour) for contour in contours]

        for bbox in bboxes:
            x, y, width, height = bbox[:4]
            
        return bbox[:4]
    
    def mouse_callback(self, event, x, y, flags, params):
        """
        Callback method for drawing character on Board
        """

        # LEFT Mouse Button is 'PRESSED'
        if event == cv2.EVENT_LBUTTONDOWN:
            self.mouse_pressed = True

        # Mouse pointer is 'MOVING'
        elif event == cv2.EVENT_MOUSEMOVE:
            if self.mouse_pressed:
                cv2.circle(img=self.board, center=(x, y), radius=5, color=self.char_color, thickness=-1)
        

        # LEFT Mouse Button is 'RELEASED'
        elif event == cv2.EVENT_LBUTTONUP:
            
            self.mouse_pressed = False
            self.count= self.count+1
            
            cnt = self.get_contours()
            
            x, y, width, height = cnt
            cv2.rectangle(self.img,(x+2,y+2),(x+width,y+height),(0,255,0),1)
            
            cv2.putText(self.img, str(self.count), (x-2, y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), lineType=cv2.LINE_AA)
            

         

In [15]:
if __name__ == '__main__':
    Board().draw()